# Machine Learning Trading Bot

### Tune the Baseline Trading Algorithm

 in this section we try to set a Date offset to 6 months,choose diffrent SMA input feature  and  fix imbalance of our target  if there any then apply a Machine Learning Classier.
 
 Lastly Create Evaluation Report 
  



In [2]:
# Imports
import pandas as pd
import numpy as np
from pathlib import Path
import hvplot.pandas
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

# Step 1:Read the OHLCV.csv dataset into a Pandas DataFrame


In [3]:
# Import the OHLCV file
Path="C:\\Users\\eszczepalink\\Downloads\\Starter_Code\\Starter_Code\\Resources\\emerging_markets_ohlcv.csv"
ohlcv_df = pd.read_csv(
    Path,
    index_col = 'date',
    infer_datetime_format = True,
    parse_dates =True
    )
# Review output
ohlcv_df

,open,high,low,close,volume
date,,,,,
2015-01-21 09:30:00,23.83,23.83,23.83,23.83,100
2015-01-21 11:00:00,23.98,23.98,23.98,23.98,100
2015-01-22 15:00:00,24.42,24.42,24.42,24.42,100
2015-01-22 15:15:00,24.42,24.44,24.42,24.44,200
2015-01-22 15:30:00,24.46,24.46,24.46,24.46,200
...,...,...,...,...,...
2021-01-22 09:30:00,33.27,33.27,33.27,33.27,100
2021-01-22 11:30:00,33.35,33.35,33.35,33.35,200
2021-01-22 13:45:00,33.42,33.42,33.42,33.42,200


In [4]:
# Keep only date index and close columns
signals_df = ohlcv_df.loc[:,["close"]]

# Apply percent change on close columns to get Actual Returns
signals_df['Actual Returns'] = signals_df["close"].pct_change()

# Drop Nan generated from apply pct_change
signals_df = signals_df.dropna()

# Review the new DataFrame
display(signals_df.head())
display(signals_df.tail())

,close,Actual Returns
date,,
2015-01-21 11:00:00,23.98,0.006295
2015-01-22 15:00:00,24.42,0.018349
2015-01-22 15:15:00,24.44,0.000819
2015-01-22 15:30:00,24.46,0.000818
2015-01-26 12:30:00,24.33,-0.005315


,close,Actual Returns
date,,
2021-01-22 09:30:00,33.27,-0.006866
2021-01-22 11:30:00,33.35,0.002405
2021-01-22 13:45:00,33.42,0.002099
2021-01-22 14:30:00,33.47,0.001496
2021-01-22 15:45:00,33.44,-0.000896


# Step 2: Generate trading signals using short and long  SMA


In [5]:
# Set the slow and fast SMA
short_window = 2
long_window = 20
# # Generate the fast and slow simple moving average (4 and 20 days, respectively)
signals_df['SMA_Fast'] =signals_df['close'].rolling(window=short_window).mean()
signals_df['SMA_Slow'] =signals_df['close'].rolling(window=long_window).mean()

# drop nans genetated from sliding windows 
signals_df = signals_df.dropna()

#Review the new DataFrame
display(signals_df.head())
display(signals_df.tail())

,close,Actual Returns,SMA_Fast,SMA_Slow
date,,,,
2015-01-30 13:30:00,23.75,-0.005860,23.820,24.1500
2015-02-02 10:15:00,23.89,0.005895,23.820,24.1455
2015-02-02 11:00:00,23.92,0.001256,23.905,24.1205
2015-02-02 11:15:00,23.94,0.000836,23.930,24.0955
2015-02-02 11:30:00,23.95,0.000418,23.945,24.0700


,close,Actual Returns,SMA_Fast,SMA_Slow
date,,,,
2021-01-22 09:30:00,33.27,-0.006866,33.385,32.4810
2021-01-22 11:30:00,33.35,0.002405,33.310,32.5955
2021-01-22 13:45:00,33.42,0.002099,33.385,32.6900
2021-01-22 14:30:00,33.47,0.001496,33.445,32.7880
2021-01-22 15:45:00,33.44,-0.000896,33.455,32.8785


In [6]:
# Create a signal column
signals_df['Signal'] = 0.0

# signal to buy stock long
signals_df.loc[(signals_df['Actual Returns'] >= 0), 'Signal'] = 1

# signal to sell stock short
signals_df.loc[(signals_df['Actual Returns'] < 0), 'Signal'] = -1

# Review the new DataFrame
display(signals_df.head())
display(signals_df.tail())

,close,Actual Returns,SMA_Fast,SMA_Slow,Signal
date,,,,,
2015-01-30 13:30:00,23.75,-0.005860,23.820,24.1500,-1.0
2015-02-02 10:15:00,23.89,0.005895,23.820,24.1455,1.0
2015-02-02 11:00:00,23.92,0.001256,23.905,24.1205,1.0
2015-02-02 11:15:00,23.94,0.000836,23.930,24.0955,1.0
2015-02-02 11:30:00,23.95,0.000418,23.945,24.0700,1.0


,close,Actual Returns,SMA_Fast,SMA_Slow,Signal
date,,,,,
2021-01-22 09:30:00,33.27,-0.006866,33.385,32.4810,-1.0
2021-01-22 11:30:00,33.35,0.002405,33.310,32.5955,1.0
2021-01-22 13:45:00,33.42,0.002099,33.385,32.6900,1.0
2021-01-22 14:30:00,33.47,0.001496,33.445,32.7880,1.0
2021-01-22 15:45:00,33.44,-0.000896,33.455,32.8785,-1.0


In [7]:
# Calculate the strategy returns then add the them to signal_df DataFrame
signals_df['Strategy Returns'] = signals_df['Actual Returns'] * signals_df['Signal'].shift()

# Review the new DataFrame
display(signals_df.head())
display(signals_df.tail())

,close,Actual Returns,SMA_Fast,SMA_Slow,Signal,Strategy Returns
date,,,,,,
2015-01-30 13:30:00,23.75,-0.005860,23.820,24.1500,-1.0,NaN
2015-02-02 10:15:00,23.89,0.005895,23.820,24.1455,1.0,-0.005895
2015-02-02 11:00:00,23.92,0.001256,23.905,24.1205,1.0,0.001256
2015-02-02 11:15:00,23.94,0.000836,23.930,24.0955,1.0,0.000836
2015-02-02 11:30:00,23.95,0.000418,23.945,24.0700,1.0,0.000418


,close,Actual Returns,SMA_Fast,SMA_Slow,Signal,Strategy Returns
date,,,,,,
2021-01-22 09:30:00,33.27,-0.006866,33.385,32.4810,-1.0,-0.006866
2021-01-22 11:30:00,33.35,0.002405,33.310,32.5955,1.0,-0.002405
2021-01-22 13:45:00,33.42,0.002099,33.385,32.6900,1.0,0.002099
2021-01-22 14:30:00,33.47,0.001496,33.445,32.7880,1.0,0.001496
2021-01-22 15:45:00,33.44,-0.000896,33.455,32.8785,-1.0,-0.000896


In [8]:
# Plot Strategy Returns to examine performance
(1 + signals_df['Strategy Returns']).cumprod().hvplot()

:Curve   [date]   (Strategy Returns)

# Step 3: Split the data into training and testing datasets


In [9]:
# X features
X = signals_df[['SMA_Fast', 'SMA_Slow']].shift().dropna().copy()

# review the DataFrame
X.head()


,SMA_Fast,SMA_Slow
date,,
2015-02-02 10:15:00,23.820,24.1500
2015-02-02 11:00:00,23.820,24.1455
2015-02-02 11:15:00,23.905,24.1205
2015-02-02 11:30:00,23.930,24.0955
2015-02-02 13:30:00,23.945,24.0700


In [10]:
# Create the target y
y = signals_df['Signal']

# Review the value counts
y.value_counts()

 1.0    2422
-1.0    1881
Name: Signal, dtype: int64

In [11]:
# training start period
training_begin = X.index.min()

# Display the training begin date
print(training_begin)

2015-02-02 10:15:00


In [12]:
# training ending period plus offset of 6 months
training_end = X.index.min() + DateOffset(months=6)

# Display the training end date
print(training_end)

2015-08-02 10:15:00


In [13]:
# Generate the X_train and y_train Dataframe
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Review the X_train DataFrame
X_train.head()

,SMA_Fast,SMA_Slow
date,,
2015-02-02 10:15:00,23.820,24.1500
2015-02-02 11:00:00,23.820,24.1455
2015-02-02 11:15:00,23.905,24.1205
2015-02-02 11:30:00,23.930,24.0955
2015-02-02 13:30:00,23.945,24.0700


In [14]:
# Generate the X_test and y_test Dataframes
X_test = X.loc[training_end:]
y_test = y.loc[training_end:]

# Reiews the X_test DataFrame
X_test.head()

,SMA_Fast,SMA_Slow
date,,
2015-08-03 09:30:00,22.980,22.8945
2015-08-03 10:15:00,22.820,22.8440
2015-08-03 10:30:00,22.630,22.7955
2015-08-03 10:45:00,22.625,22.7510
2015-08-03 11:00:00,22.660,22.7205


In [15]:
#Scale the features DataFrame

# Create a StandardScaler instance
scaler = StandardScaler()

# Apply the scaler model of fit the X_train dta

X_scaler = scaler.fit(X_train)

# Transform the X_train and X_test Dataframes using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Review the new DataFrame
display(X_train_scaled[:10])
display(X_test_scaled)

array([[-0.75833283, -0.52979868],
       [-0.75833283, -0.53496222],
       [-0.67149374, -0.56364853],
       [-0.64595283, -0.59233484],
       [-0.63062828, -0.62159487],
       [-0.66127737, -0.64741255],
       [-0.55400555, -0.65429726],
       [-0.40076009, -0.65888707],
       [-0.39565191, -0.66290315],
       [-0.41097646, -0.65716589]])

array([[-1.6165074 , -1.97042505],
       [-1.77996923, -2.02837139],
       [-1.97408014, -2.08402283],
       ...,
       [ 8.93699654,  9.16100975],
       [ 9.01361927,  9.26944399],
       [ 9.07491746,  9.38189432]])

In [16]:
# review value counts of target y
y.value_counts()

 1.0    2422
-1.0    1881
Name: Signal, dtype: int64

In [17]:
# fix imbalances of our target y
#import model 
ros = RandomOverSampler()
# fit model
X_train_scaled_ros, y_train_ros = ros.fit_resample(X_train_scaled,y_train)

# check the model 
Counter(y_train_ros)

Counter({1.0: 152, -1.0: 152})

# Step 4: using SVC classifier model, SVM learning method to fit, train and predict based on the testing data


In [18]:
# From SVM, instantiate SVC classifier model instance
svm_model = SVC()
 
# Fit the model to the data using the training data
svm_model = svm_model.fit(X_train_scaled_ros,y_train_ros)
 
# Use the testing data to make the model predictions
svm_pred = svm_model.predict(X_test_scaled)

# Review the model's predicted values
svm_pred[:10]


array([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1.])

 ### Step 5: Review the classification report associated with the `SVC` model predictions. 

In [19]:
# Use a classification report to evaluate the model using the predictions and testing data
svm_testing_report = classification_report(y_test, svm_pred)

# Print the classification report
print(svm_testing_report)

              precision    recall  f1-score   support

        -1.0       0.44      0.72      0.54      1779
         1.0       0.56      0.28      0.37      2270

    accuracy                           0.47      4049
   macro avg       0.50      0.50      0.46      4049
weighted avg       0.50      0.47      0.45      4049



### Step 6: Create a predictions DataFrame that contains columns for “Predicted” values, “Actual Returns”, and “Strategy Returns”.

In [20]:
# Create a new empty predictions DataFrame:

# Create a predictions DataFrame
predictions_df = pd.DataFrame({
    'Predited':svm_pred,
    'Actual Returns':signals_df['Actual Returns'],
    'Strategy Returns':signals_df['Strategy Returns']
}, index = X_test.index)

# Add the SVM model predictions to the DataFrame
predictions_df['Predicted'] = svm_pred

# Add the actual returns to the DataFrame
predictions_df['Actual Returns'] = signals_df['Actual Returns']

# Add the strategy returns to the DataFrame
predictions_df['Strategy Returns'] = signals_df['Strategy Returns']

# Review the DataFrame
display(predictions_df.head())
display(predictions_df.tail())

,Predited,Actual Returns,Strategy Returns,Predicted
date,,,,
2015-08-03 09:30:00,-1.0,-0.013925,-0.013925,-1.0
2015-08-03 10:15:00,-1.0,-0.002648,0.002648,-1.0
2015-08-03 10:30:00,-1.0,0.002212,-0.002212,-1.0
2015-08-03 10:45:00,-1.0,0.000883,0.000883,-1.0
2015-08-03 11:00:00,-1.0,0.000441,0.000441,-1.0


,Predited,Actual Returns,Strategy Returns,Predicted
date,,,,
2021-01-22 09:30:00,-1.0,-0.006866,-0.006866,-1.0
2021-01-22 11:30:00,-1.0,0.002405,-0.002405,-1.0
2021-01-22 13:45:00,-1.0,0.002099,0.002099,-1.0
2021-01-22 14:30:00,-1.0,0.001496,0.001496,-1.0
2021-01-22 15:45:00,-1.0,-0.000896,-0.000896,-1.0


### Step 7: Create a cumulative return plot that shows the actual returns vs. the strategy returns. Save a PNG image of this plot. This will serve as a baseline against which to compare the effects of tuning the trading algorithm.

In [21]:
# Plot the actual returns versus the strategy returns
(1+predictions_df[['Actual Returns','Strategy Returns']]).cumprod().hvplot()

:NdOverlay   [Variable]
   :Curve   [date]   (value)

## Evaluate a New Machine Learning Classifier

In this section, you’ll use the original parameters that the starter code provided. But, you’ll apply them to the performance of a second machine learning model. 

In [22]:
# Import a new classifier from SKLearn
from sklearn.tree import DecisionTreeClassifier

# Initiate the model instance
model_tree = DecisionTreeClassifier()

### Step 2: Using the original training data as the baseline model, fit another model with the new classifier.

In [23]:
# Fit the model using the training data
model = model_tree.fit(X_train_scaled_ros,y_train_ros)

# Use the testing dataset to generate the predictions for the new model
pred = model.predict(X_test_scaled)

# Review the model's predicted values
pred


array([1., 1., 1., ..., 1., 1., 1.])

### Step 3: Backtest the new model to evaluate its performance. 

Save a PNG image of the cumulative product of the actual returns vs. the strategy returns for this updated trading algorithm, and write your conclusions in your `README.md` file. 

Answer the following questions: 
Did this new model perform better or worse than the provided baseline model? 
Did this new model perform better or worse than your tuned trading algorithm?

In [24]:
# Use a classification report to evaluate the model using the predictions and testing data
classification_report(y_test, pred)

# Print the classification report
print(classification_report(y_test, pred))


              precision    recall  f1-score   support

        -1.0       0.44      0.23      0.30      1779
         1.0       0.56      0.77      0.65      2270

    accuracy                           0.53      4049
   macro avg       0.50      0.50      0.48      4049
weighted avg       0.51      0.53      0.50      4049



In [25]:
'''

The new model acted better than previous one gave us 54% of accuracy compared to 48%

The changing of the sliding window buy and sell, offset date and making a balance in our target helped to improve 
the accuracy of baseline model to trading algothrithmic 

'''

'\n\nThe new model acted better than previous one gave us 54% of accuracy compared to 48%\n\nThe changing of the sliding window buy and sell, offset date and making a balance in our target helped to improve \nthe accuracy of baseline model to trading algothrithmic \n\n'

In [26]:
# Create a new empty predictions DataFrame:

predictions = pd.DataFrame({
    "actual value": y_test,
    "predicted value": pred,
    'Predited':svm_pred,
    'Actual Returns':signals_df['Actual Returns'],
    'Strategy Returns':signals_df['Strategy Returns']
}, index = X_test.index)

    


# Review the DataFrame
predictions


,actual value,predicted value,Predited,Actual Returns,Strategy Returns
date,,,,,
2015-08-03 09:30:00,-1.0,1.0,-1.0,-0.013925,-0.013925
2015-08-03 10:15:00,-1.0,1.0,-1.0,-0.002648,0.002648
2015-08-03 10:30:00,1.0,1.0,-1.0,0.002212,-0.002212
2015-08-03 10:45:00,1.0,1.0,-1.0,0.000883,0.000883
2015-08-03 11:00:00,1.0,1.0,-1.0,0.000441,0.000441
...,...,...,...,...,...
2021-01-22 09:30:00,-1.0,1.0,-1.0,-0.006866,-0.006866
2021-01-22 11:30:00,1.0,1.0,-1.0,0.002405,-0.002405
2021-01-22 13:45:00,1.0,1.0,-1.0,0.002099,0.002099


In [27]:
# Plot the actual returns versus the strategy returns
(1+predictions[['Actual Returns','Strategy Returns']]).cumprod().hvplot()

:NdOverlay   [Variable]
   :Curve   [date]   (value)